# Acceleroseismology: A model for identifying FoG in Parkinson's Disease patients.

## Background:

Freezing of Gait is a symptom of Parkinson's Disease (PD) that is debilitating, lowers quality of life, reduces independence, and can lead to an increased risk of falling.  In an effort to identify Freezing of Gait signatures in accelerometer data (or better yet, predict occurences ahead of time), this notebook applies a machine learning algorithm to the time series accelerometry.  I hope the results are useful in diagnosing and potentially warning PD patients ahead of Freezing of Gait (FoG) episodes.


## Acceleroseismology:

As a mental model, I have approached this problem similar to seismology.  In studying earthquakes, seismologists discovered that P-waves arrive ahead of more destructive S-waves, allowing for an early warning system to avoid adverse effects.  Similarly, in what I am calling acceleroseismology, I analyze both time and frequency domain features in accelerometry data via a moving window approach in order to place each instance of time in its broader context.  Ultimately, I hope to engineer relevant features that can predict the coming onset of a FoG episode, as such a capability could enable wearable devices to alert the user and possibly reduce fall risk and improve the probability of avoiding the FoG event altogether.  Such a result would contribute to the United Nations' [Third Sustainable Development Goal: Good health and well-being](https://sdgs.un.org/goals).


## Motivation Behind Features:



In [1]:
# Import Libraries:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
#import random
import csv
#import matplotlib.pyplot as plt
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
#from sklearn.neural_network import MLPClassifier
#from sklearn.ensemble import RandomForestRegressor
#from sklearn.metrics import mean_absolute_error
#from sklearn.model_selection import train_test_split


####################################################################
########               PARAMETERS & CONSTANTS:               ########
####################################################################
sampleRateTDCSFOG = 128     #per second
sampleRateDEFOG = 100     #per second
windowHalfLength = 21 #on either side of the point of interest (for the moving window)
peakThreshold = 1.4  #peakThreshold*mean value identifies a peak

#low frequency band
lowfBandMin = 0.0
lowfBandMax = 2.0
#high frequency band
highfBandMin = 2.0
highfBandMax = 6.0

dummyVariable = 9 #ignore this for now, please.



**Functions:**

In [2]:
#A fxn that computes the root mean abs square value of an array.
def getRMS(inputArray):
    return np.sqrt(np.mean(abs(inputArray)*abs(inputArray))) #imaginary args okay


#A low pass filter to remove high frequency noise.
def lowPassFilter(kArr, freqArr, cutOffFreq):
    for i in range(0,len(freqArr)):
        if freqArr[i] > cutOffFreq:
            kArr.real[i] = 0; 
            kArr.imag[i] = 0;
    return kArr


#A high pass filter to analyze only high frequencies.  
def highPassFilter(kArr, freqArr, cutOffFreq):
    for i in range(0,len(freqArr)):
        if freqArr[i] < cutOffFreq:
            kArr.real[i] = 0;
            kArr.imag[i] = 0;
    return kArr


#A quick FFT where W can be x, y, z accelerations etc.
def quickFFT(inputT, inputW, sampleRate, filterType, cutOff):
    kspaceData = np.fft.rfft(inputW)
    freq = np.fft.rfftfreq(inputT.shape[-1], d=1.0/sampleRate)
    if filterType == "low":
        filteredData = lowPassFilter(kspaceData, freq, cutOff)
    elif filterType == "high":
        filteredData = highPassFilter(kspaceData, freq, cutOff)
    else:
        filteredData = kspaceData
    outputW = np.fft.irfft(filteredData, len(inputW))
    return outputW


#A quick FFT where W can be x, y, z accelerations etc. (returns k-space)
def quickFFT_k(inputT, inputW, sampleRate, filterType, cutOff):
    kspaceData = np.fft.rfft(inputW)
    freq = np.fft.rfftfreq(inputT.shape[-1], d=1.0/sampleRate)
    if filterType == "low":
        filteredData = lowPassFilter(kspaceData, freq, cutOff)
    elif filterType == "high":
        filteredData = highPassFilter(kspaceData, freq, cutOff)
    else:
        filteredData = kspaceData
    return freq, filteredData

**Read in training data:**


In [3]:
# Load the training and testing data:
#train_file_path = '/kaggle/input/parkinsonsfog-newfeatures-traintest/trainFinal.csv'
#test_file_path = '/kaggle/input/parkinsonsfog-newfeatures-traintest/testFinal.csv'

trainTDCSFOG_path = '/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/train/tdcsfog/'
trainDEFOG_path = '/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/train/defog/'

testTDCSFOG_path = '/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/test/tdcsfog/'
testDEFOG_path = '/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/test/defog/'


#print("Reading training file...")
#trainTDCSFOG_data = pd.read_csv(trainTDCSFOG_path)
#test_data = pd.read_csv(test_file_path)

**Process data:**

In [4]:
colNames_TDCS = ['Time', 'aVert', 'aML', 'aAP', 'StartHesitation', 'Turn', 'Walking', 'id_t']
colNames_DEFOG = ['Time', 'aVert', 'aML', 'aAP', 'StartHesitation', 'Turn', 'Walking', 'Valid', 'Task', 'id_t']

trainTDCS_List = []
trainDEFOG_List = []

#Process data:
print('Creating TDCS list...')
for dirname, _, filenames in os.walk(trainTDCSFOG_path):
    for filename in filenames:
        #print(my_files_path+filename)
        id = filename[:-4] #removes trailing .csv
        f_trainTDCS_List = [] #file specific list, add elements to main list later
        with open(trainTDCSFOG_path+filename) as file_obj:
            heading = next(file_obj)
            reader_obj = csv.reader(file_obj)   
            # Iterate over each row in the csv file:
            for row in reader_obj:
                id_t = id + "_" + str(row[0])
                row.append(id_t)
                f_trainTDCS_List.append(row)
        #File specific list is f_trainTDCS_List
        #Feature engineering now begins for the file:
        numPoints = len(f_trainTDCS_List)
        dfTemp = pd.DataFrame(f_trainTDCS_List,columns=colNames_TDCS)
        t = np.array(dfTemp['Time'].astype('float64'))
        aML = np.array(dfTemp['aML'].astype('float64'))
        aAP = np.array(dfTemp['aAP'].astype('float64'))
        aVert = np.array(dfTemp['aVert'].astype('float64'))
        boolStartHes = np.array(dfTemp['StartHesitation'])
        boolTurn = np.array(dfTemp['Turn'])
        boolWalking = np.array(dfTemp['Walking'])  
        id_t = dfTemp['id_t']
        #Initialize arrays:
        maxAmpAP         = np.zeros(numPoints) # abs val of maximum amplitude in FFT'd AP data
        maxAccelAmpML    = np.zeros(numPoints) # abs(max - min accel) for ML accel data
        maxAccelAmpAP    = np.zeros(numPoints) # abs(max - min accel) for AP accel data
        maxAccelAmpVert  = np.zeros(numPoints) # abs(max - min accel) for Vert accel data
        rmsAmpsLowfAP    = np.zeros(numPoints) # root-mean-square (rms) of amplitudes in low freq band (AP)
        rmsAmpsLowfVert  = np.zeros(numPoints) # root-mean-square (rms) of amplitudes in low freq band (vert)
        rmsAmpsHighfML   = np.zeros(numPoints) # root-mean-square (rms) of amplitudes in high freq band (ML)
        for tm in range(windowHalfLength, numPoints-windowHalfLength-1):
            tClip = t[tm-windowHalfLength:tm+windowHalfLength+1]
            aML_Clip = aML[tm-windowHalfLength:tm+windowHalfLength+1]
            aAP_Clip = aAP[tm-windowHalfLength:tm+windowHalfLength+1]
            aVert_Clip = aVert[tm-windowHalfLength:tm+windowHalfLength+1]
            freqML, ampsML = quickFFT_k(tClip, aML_Clip, sampleRateTDCSFOG, "none", dummyVariable)
            freqAP, ampsAP = quickFFT_k(tClip, aAP_Clip, sampleRateTDCSFOG, "none", dummyVariable)
            freqVert, ampsVert = quickFFT_k(tClip, aVert_Clip, sampleRateTDCSFOG, "none", dummyVariable)
            meanAccelML   = np.nanmean(abs(aML_Clip))     #Mean abs acceleration in the ML direction during time window
            meanAccelAP   = np.nanmean(abs(aAP_Clip))     #Mean abs acceleration in the AP direction during time window
            meanAccelVert = np.nanmean(abs(aVert_Clip))   #Mean abs acceleration in the Vert direction during time window
            absAmpsML     = abs(ampsML)
            absAmpsAP     = abs(ampsAP)
            absAmpsVert   = abs(ampsVert)
            meanAmpML     = np.nanmean(absAmpsML)       #Mean abs amplitude for ML data (FFT'd)
            meanAmpAP     = np.nanmean(absAmpsAP)       #Mean abs amplitude for AP data (FFT'd)
            meanAmpVert   = np.nanmean(absAmpsVert)     #Mean abs amplitude for Vert data (FFT'd)
            #Start Filling Arrays:
            maxAmpAP[tm] = np.amax(absAmpsAP,axis=0)     # abs val of maximum amplitude in FFT'd AP data
            maxAccelAmpML[tm] = np.max(abs(aML_Clip)) - np.min(abs(aML_Clip))       # abs(max - min accel) for ML accel data
            maxAccelAmpAP[tm] = np.max(abs(aAP_Clip)) - np.min(abs(aAP_Clip))       # abs(max - min accel) for AP accel data
            maxAccelAmpVert[tm] = np.max(abs(aVert_Clip)) - np.min(abs(aVert_Clip)) # abs(max - min accel) for Vert accel data
            rmsAmpsLowfAP[tm] = getRMS(ampsAP[freqAP<lowfBandMax])
            rmsAmpsLowfVert[tm] = getRMS(ampsVert[freqVert<lowfBandMax])
            rmsAmpsHighfML[tm] = getRMS(ampsML[(freqML>highfBandMin)*(freqML<highfBandMax)])
        for dt in range(0, numPoints):  
            addThis = []
            addThis.append(maxAmpAP[dt])
            addThis.append(maxAccelAmpML[dt])
            addThis.append(maxAccelAmpAP[dt])
            addThis.append(maxAccelAmpVert[dt])
            addThis.append(rmsAmpsLowfAP[dt])
            addThis.append(rmsAmpsLowfVert[dt])
            addThis.append(rmsAmpsHighfML[dt])
            #Finally, add the bools to the end:
            addThis.append(boolStartHes[dt])
            addThis.append(boolTurn[dt])
            addThis.append(boolWalking[dt])
            addThis.append(id_t[dt])
            trainTDCS_List.append(addThis)

            
print('Creating DEFOG list...')
for dirname, _, filenames in os.walk(trainDEFOG_path):
    for filename in filenames:
        #print(my_files_path+filename)
        id = filename[:-4] #removes trailing .csv
        f_trainDEFOG_List = [] #file specific list, add elements to main list later
        with open(trainDEFOG_path+filename) as file_obj:
            heading = next(file_obj)
            reader_obj = csv.reader(file_obj)   
            # Iterate over each row in the csv file:
            for row in reader_obj:
                id_t = id + "_" + str(row[0])
                row.append(id_t)
                f_trainDEFOG_List.append(row)
        #File specific list is f_trainTDCS_List
        #Feature engineering now begins for the file:
        numPoints = len(f_trainDEFOG_List)
        dfTemp = pd.DataFrame(f_trainDEFOG_List,columns=colNames_DEFOG)
        t = np.array(dfTemp['Time'].astype('float64'))
        aML = np.array(dfTemp['aML'].astype('float64'))
        aAP = np.array(dfTemp['aAP'].astype('float64'))
        aVert = np.array(dfTemp['aVert'].astype('float64'))
        boolStartHes = np.array(dfTemp['StartHesitation'])
        boolTurn = np.array(dfTemp['Turn'])
        boolWalking = np.array(dfTemp['Walking'])  
        id_t = dfTemp['id_t']
        #Initialize arrays:
        maxAmpAP         = np.zeros(numPoints) # abs val of maximum amplitude in FFT'd AP data
        maxAccelAmpML    = np.zeros(numPoints) # abs(max - min accel) for ML accel data
        maxAccelAmpAP    = np.zeros(numPoints) # abs(max - min accel) for AP accel data
        maxAccelAmpVert  = np.zeros(numPoints) # abs(max - min accel) for Vert accel data
        rmsAmpsLowfAP    = np.zeros(numPoints) # root-mean-square (rms) of amplitudes in low freq band (AP)
        rmsAmpsLowfVert  = np.zeros(numPoints) # root-mean-square (rms) of amplitudes in low freq band (vert)
        rmsAmpsHighfML   = np.zeros(numPoints) # root-mean-square (rms) of amplitudes in high freq band (ML)
        for tm in range(windowHalfLength, numPoints-windowHalfLength-1):
            tClip = t[tm-windowHalfLength:tm+windowHalfLength+1]
            aML_Clip = aML[tm-windowHalfLength:tm+windowHalfLength+1]
            aAP_Clip = aAP[tm-windowHalfLength:tm+windowHalfLength+1]
            aVert_Clip = aVert[tm-windowHalfLength:tm+windowHalfLength+1]
            freqML, ampsML = quickFFT_k(tClip, aML_Clip, sampleRateDEFOG, "none", dummyVariable)
            freqAP, ampsAP = quickFFT_k(tClip, aAP_Clip, sampleRateDEFOG, "none", dummyVariable)
            freqVert, ampsVert = quickFFT_k(tClip, aVert_Clip, sampleRateDEFOG, "none", dummyVariable)
            meanAccelML   = np.nanmean(abs(aML_Clip))     #Mean abs acceleration in the ML direction during time window
            meanAccelAP   = np.nanmean(abs(aAP_Clip))     #Mean abs acceleration in the AP direction during time window
            meanAccelVert = np.nanmean(abs(aVert_Clip))   #Mean abs acceleration in the Vert direction during time window
            absAmpsML     = abs(ampsML)
            absAmpsAP     = abs(ampsAP)
            absAmpsVert   = abs(ampsVert)
            meanAmpML     = np.nanmean(absAmpsML)       #Mean abs amplitude for ML data (FFT'd)
            meanAmpAP     = np.nanmean(absAmpsAP)       #Mean abs amplitude for AP data (FFT'd)
            meanAmpVert   = np.nanmean(absAmpsVert)     #Mean abs amplitude for Vert data (FFT'd)
            #Start Filling Arrays:
            maxAmpAP[tm] = np.amax(absAmpsAP,axis=0)     # abs val of maximum amplitude in FFT'd AP data
            maxAccelAmpML[tm] = np.max(abs(aML_Clip)) - np.min(abs(aML_Clip))       # abs(max - min accel) for ML accel data
            maxAccelAmpAP[tm] = np.max(abs(aAP_Clip)) - np.min(abs(aAP_Clip))       # abs(max - min accel) for AP accel data
            maxAccelAmpVert[tm] = np.max(abs(aVert_Clip)) - np.min(abs(aVert_Clip)) # abs(max - min accel) for Vert accel data
            rmsAmpsLowfAP[tm] = getRMS(ampsAP[freqAP<lowfBandMax])
            rmsAmpsLowfVert[tm] = getRMS(ampsVert[freqVert<lowfBandMax])
            rmsAmpsHighfML[tm] = getRMS(ampsML[(freqML>highfBandMin)*(freqML<highfBandMax)])
        for dt in range(0, numPoints):  
            addThis = []
            addThis.append(maxAmpAP[dt])
            addThis.append(maxAccelAmpML[dt])
            addThis.append(maxAccelAmpAP[dt])
            addThis.append(maxAccelAmpVert[dt])
            addThis.append(rmsAmpsLowfAP[dt])
            addThis.append(rmsAmpsLowfVert[dt])
            addThis.append(rmsAmpsHighfML[dt])
            #Finally, add the bools to the end:
            addThis.append(boolStartHes[dt])
            addThis.append(boolTurn[dt])
            addThis.append(boolWalking[dt])
            addThis.append(id_t[dt])
            trainDEFOG_List.append(addThis)






Creating TDCS list...
Creating DEFOG list...


**Train the models:**

In [5]:
features = ['maxAmpAP', 'maxAccelAmpML', 'maxAccelAmpAP', 'maxAccelAmpVert', 'rmsAmpsLowfAP', 'rmsAmpsLowfVert', 'rmsAmpsHighfML']

outputCols = ['StartHesitation', 'Turn', 'Walking']


newColNames_TDCS = ['maxAmpAP', 'maxAccelAmpML', 'maxAccelAmpAP', 'maxAccelAmpVert', 'rmsAmpsLowfAP', 'rmsAmpsLowfVert', 'rmsAmpsHighfML', 'StartHesitation', 'Turn', 'Walking', 'id_t']
newColNames_DEFOG = ['maxAmpAP', 'maxAccelAmpML', 'maxAccelAmpAP', 'maxAccelAmpVert', 'rmsAmpsLowfAP', 'rmsAmpsLowfVert', 'rmsAmpsHighfML', 'StartHesitation', 'Turn', 'Walking', 'id_t']


                
print('Creating dataframes...')
dfTrain_TDCS = pd.DataFrame(trainTDCS_List, columns = newColNames_TDCS)
dfTrain_DEFOG = pd.DataFrame(trainDEFOG_List, columns = newColNames_DEFOG)





#TDCS
print("Training the TDCS model...")
X_TDCS = dfTrain_TDCS.loc[:, features] 
y_TDCS = dfTrain_TDCS.loc[:, outputCols]
clf_TDCS = MultiOutputClassifier(LogisticRegression(max_iter=50000)).fit(X_TDCS, y_TDCS)






#DEFOG
print("Training the DEFOG model...")
X_DEFOG = dfTrain_DEFOG.loc[:, features] 
y_DEFOG = dfTrain_DEFOG.loc[:, outputCols]
clf_DEFOG = MultiOutputClassifier(LogisticRegression(max_iter=50000)).fit(X_DEFOG, y_DEFOG)



Creating dataframes...
Training the TDCS model...
Training the DEFOG model...


**Now Test the model:**

In [6]:
#Read in test data
print("Reading testing files...")
#test_file_path_TDCS = '/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/test/tdcsfog/003f117e14.csv'
#test_file_path_DEFOG = '/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/test/defog/02ab235146.csv'

testTDCS_path = '/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/test/tdcsfog/'
testDEFOG_path = '/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/test/defog/'

testTDCS_List = []
testDEFOG_List = []

print('Creating TDCS test list...')
for dirname, _, filenames in os.walk(testTDCS_path):
    for filename in filenames:
        #print(my_files_path+filename)
        id = filename[:-4] #removes trailing .csv
        f_testTDCS_List = [] #file specific list, add elements to main list later
        with open(testTDCS_path+filename) as file_obj:
            heading = next(file_obj)
            reader_obj = csv.reader(file_obj)   
            # Iterate over each row in the csv file:
            for row in reader_obj:
                id_t = id + "_" + str(row[0])
                row.append(id_t)
                f_testTDCS_List.append(row)
        #Feature engineering now begins for the file:
        numPoints = len(f_testTDCS_List)
        dfTemp = pd.DataFrame(f_testTDCS_List,columns=['Time', 'aVert', 'aML', 'aAP', 'id_t'])
        t = np.array(dfTemp['Time'].astype('float64'))
        aML = np.array(dfTemp['aML'].astype('float64'))
        aAP = np.array(dfTemp['aAP'].astype('float64'))
        aVert = np.array(dfTemp['aVert'].astype('float64'))
        id_t = dfTemp['id_t']
        #Initialize arrays:
        maxAmpAP         = np.zeros(numPoints) # abs val of maximum amplitude in FFT'd AP data
        maxAccelAmpML    = np.zeros(numPoints) # abs(max - min accel) for ML accel data
        maxAccelAmpAP    = np.zeros(numPoints) # abs(max - min accel) for AP accel data
        maxAccelAmpVert  = np.zeros(numPoints) # abs(max - min accel) for Vert accel data
        rmsAmpsLowfAP    = np.zeros(numPoints) # root-mean-square (rms) of amplitudes in low freq band (AP)
        rmsAmpsLowfVert  = np.zeros(numPoints) # root-mean-square (rms) of amplitudes in low freq band (vert)
        rmsAmpsHighfML   = np.zeros(numPoints) # root-mean-square (rms) of amplitudes in high freq band (ML)
        for tm in range(windowHalfLength, numPoints-windowHalfLength-1):
            tClip = t[tm-windowHalfLength:tm+windowHalfLength+1]
            aML_Clip = aML[tm-windowHalfLength:tm+windowHalfLength+1]
            aAP_Clip = aAP[tm-windowHalfLength:tm+windowHalfLength+1]
            aVert_Clip = aVert[tm-windowHalfLength:tm+windowHalfLength+1]
            freqML, ampsML = quickFFT_k(tClip, aML_Clip, sampleRateTDCSFOG, "none", dummyVariable)
            freqAP, ampsAP = quickFFT_k(tClip, aAP_Clip, sampleRateTDCSFOG, "none", dummyVariable)
            freqVert, ampsVert = quickFFT_k(tClip, aVert_Clip, sampleRateTDCSFOG, "none", dummyVariable)
            meanAccelML   = np.nanmean(abs(aML_Clip))     #Mean abs acceleration in the ML direction during time window
            meanAccelAP   = np.nanmean(abs(aAP_Clip))     #Mean abs acceleration in the AP direction during time window
            meanAccelVert = np.nanmean(abs(aVert_Clip))   #Mean abs acceleration in the Vert direction during time window
            absAmpsML     = abs(ampsML)
            absAmpsAP     = abs(ampsAP)
            absAmpsVert   = abs(ampsVert)
            meanAmpML     = np.nanmean(absAmpsML)       #Mean abs amplitude for ML data (FFT'd)
            meanAmpAP     = np.nanmean(absAmpsAP)       #Mean abs amplitude for AP data (FFT'd)
            meanAmpVert   = np.nanmean(absAmpsVert)     #Mean abs amplitude for Vert data (FFT'd)
            #Start Filling Arrays:
            maxAmpAP[tm] = np.amax(absAmpsAP,axis=0)     # abs val of maximum amplitude in FFT'd AP data
            maxAccelAmpML[tm] = np.max(abs(aML_Clip)) - np.min(abs(aML_Clip))       # abs(max - min accel) for ML accel data
            maxAccelAmpAP[tm] = np.max(abs(aAP_Clip)) - np.min(abs(aAP_Clip))       # abs(max - min accel) for AP accel data
            maxAccelAmpVert[tm] = np.max(abs(aVert_Clip)) - np.min(abs(aVert_Clip)) # abs(max - min accel) for Vert accel data
            rmsAmpsLowfAP[tm] = getRMS(ampsAP[freqAP<lowfBandMax])
            rmsAmpsLowfVert[tm] = getRMS(ampsVert[freqVert<lowfBandMax])
            rmsAmpsHighfML[tm] = getRMS(ampsML[(freqML>highfBandMin)*(freqML<highfBandMax)])
        for dt in range(0, numPoints):  
            addThis = []
            addThis.append(maxAmpAP[dt])
            addThis.append(maxAccelAmpML[dt])
            addThis.append(maxAccelAmpAP[dt])
            addThis.append(maxAccelAmpVert[dt])
            addThis.append(rmsAmpsLowfAP[dt])
            addThis.append(rmsAmpsLowfVert[dt])
            addThis.append(rmsAmpsHighfML[dt])
            addThis.append(id_t[dt])
            testTDCS_List.append(addThis)



print('Creating DEFOG test list...')
for dirname, _, filenames in os.walk(testDEFOG_path):
    for filename in filenames:
        #print(my_files_path+filename)
        id = filename[:-4] #removes trailing .csv
        f_testDEFOG_List = [] #file specific list, add elements to main list later
        with open(testDEFOG_path+filename) as file_obj:
            heading = next(file_obj)
            reader_obj = csv.reader(file_obj)   
            # Iterate over each row in the csv file:
            for row in reader_obj:
                id_t = id + "_" + str(row[0])
                row.append(id_t)
                f_testDEFOG_List.append(row)
        #Feature engineering now begins for the file:
        numPoints = len(f_testDEFOG_List)
        dfTemp = pd.DataFrame(f_testDEFOG_List,columns=['Time', 'aVert', 'aML', 'aAP', 'id_t'])
        t = np.array(dfTemp['Time'].astype('float64'))
        aML = np.array(dfTemp['aML'].astype('float64'))
        aAP = np.array(dfTemp['aAP'].astype('float64'))
        aVert = np.array(dfTemp['aVert'].astype('float64')) 
        id_t = dfTemp['id_t']
        #Initialize arrays:
        maxAmpAP         = np.zeros(numPoints) # abs val of maximum amplitude in FFT'd AP data
        maxAccelAmpML    = np.zeros(numPoints) # abs(max - min accel) for ML accel data
        maxAccelAmpAP    = np.zeros(numPoints) # abs(max - min accel) for AP accel data
        maxAccelAmpVert  = np.zeros(numPoints) # abs(max - min accel) for Vert accel data
        rmsAmpsLowfAP    = np.zeros(numPoints) # root-mean-square (rms) of amplitudes in low freq band (AP)
        rmsAmpsLowfVert  = np.zeros(numPoints) # root-mean-square (rms) of amplitudes in low freq band (vert)
        rmsAmpsHighfML   = np.zeros(numPoints) # root-mean-square (rms) of amplitudes in high freq band (ML)
        for tm in range(windowHalfLength, numPoints-windowHalfLength-1):
            tClip = t[tm-windowHalfLength:tm+windowHalfLength+1]
            aML_Clip = aML[tm-windowHalfLength:tm+windowHalfLength+1]
            aAP_Clip = aAP[tm-windowHalfLength:tm+windowHalfLength+1]
            aVert_Clip = aVert[tm-windowHalfLength:tm+windowHalfLength+1]
            freqML, ampsML = quickFFT_k(tClip, aML_Clip, sampleRateDEFOG, "none", dummyVariable)
            freqAP, ampsAP = quickFFT_k(tClip, aAP_Clip, sampleRateDEFOG, "none", dummyVariable)
            freqVert, ampsVert = quickFFT_k(tClip, aVert_Clip, sampleRateDEFOG, "none", dummyVariable)
            meanAccelML   = np.nanmean(abs(aML_Clip))     #Mean abs acceleration in the ML direction during time window
            meanAccelAP   = np.nanmean(abs(aAP_Clip))     #Mean abs acceleration in the AP direction during time window
            meanAccelVert = np.nanmean(abs(aVert_Clip))   #Mean abs acceleration in the Vert direction during time window
            absAmpsML     = abs(ampsML)
            absAmpsAP     = abs(ampsAP)
            absAmpsVert   = abs(ampsVert)
            meanAmpML     = np.nanmean(absAmpsML)       #Mean abs amplitude for ML data (FFT'd)
            meanAmpAP     = np.nanmean(absAmpsAP)       #Mean abs amplitude for AP data (FFT'd)
            meanAmpVert   = np.nanmean(absAmpsVert)     #Mean abs amplitude for Vert data (FFT'd)
            #Start Filling Arrays:
            maxAmpAP[tm] = np.amax(absAmpsAP,axis=0)     # abs val of maximum amplitude in FFT'd AP data
            maxAccelAmpML[tm] = np.max(abs(aML_Clip)) - np.min(abs(aML_Clip))       # abs(max - min accel) for ML accel data
            maxAccelAmpAP[tm] = np.max(abs(aAP_Clip)) - np.min(abs(aAP_Clip))       # abs(max - min accel) for AP accel data
            maxAccelAmpVert[tm] = np.max(abs(aVert_Clip)) - np.min(abs(aVert_Clip)) # abs(max - min accel) for Vert accel data
            rmsAmpsLowfAP[tm] = getRMS(ampsAP[freqAP<lowfBandMax])
            rmsAmpsLowfVert[tm] = getRMS(ampsVert[freqVert<lowfBandMax])
            rmsAmpsHighfML[tm] = getRMS(ampsML[(freqML>highfBandMin)*(freqML<highfBandMax)])
        for dt in range(0, numPoints):  
            addThis = []
            addThis.append(maxAmpAP[dt])
            addThis.append(maxAccelAmpML[dt])
            addThis.append(maxAccelAmpAP[dt])
            addThis.append(maxAccelAmpVert[dt])
            addThis.append(rmsAmpsLowfAP[dt])
            addThis.append(rmsAmpsLowfVert[dt])
            addThis.append(rmsAmpsHighfML[dt])
            #Finally, add the bools to the end:
            addThis.append(id_t[dt])
            testDEFOG_List.append(addThis)
            
            

            
print('Creating dataframes...')
dfTest_TDCS = pd.DataFrame(testTDCS_List, columns = ['maxAmpAP', 'maxAccelAmpML', 'maxAccelAmpAP', 'maxAccelAmpVert', 'rmsAmpsLowfAP', 'rmsAmpsLowfVert', 'rmsAmpsHighfML', 'id_t'])
dfTest_DEFOG = pd.DataFrame(testDEFOG_List, columns = ['maxAmpAP', 'maxAccelAmpML', 'maxAccelAmpAP', 'maxAccelAmpVert', 'rmsAmpsLowfAP', 'rmsAmpsLowfVert', 'rmsAmpsHighfML', 'id_t'])



#TDCS
X_val_TDCS = dfTest_TDCS.loc[:, features]
print("Making TDCS predictions...")
clf_predictions_TDCS = clf_TDCS.predict(X_val_TDCS)
df_clf_predictions_TDCS = pd.DataFrame(clf_predictions_TDCS, columns=outputCols)



#DEFOG:
X_val_DEFOG = dfTest_DEFOG.loc[:, features]
print("Making DEFOG predictions...")
clf_predictions_DEFOG = clf_DEFOG.predict(X_val_DEFOG)
df_clf_predictions_DEFOG = pd.DataFrame(clf_predictions_DEFOG, columns=outputCols)








Reading testing files...
Creating TDCS test list...
Creating DEFOG test list...
Creating dataframes...
Making TDCS predictions...
Making DEFOG predictions...


In [7]:
#store df of submission results.
#finalList = [[] for _ in range(0, len(rf_val_predictions_Turn))]
#or do df since we're just zipping?
finalCols = ['id', 'StartHesitation', 'Turn', 'Walking']

id_t_TDCS = dfTest_TDCS.loc[:, 'id_t']
id_t_DEFOG = dfTest_TDCS.loc[:, 'id_t']

df_clf_predictions_TDCS['id'] =  id_t_TDCS
df_clf_predictions_DEFOG['id'] = id_t_DEFOG

df_TDCS = df_clf_predictions_TDCS[finalCols]
df_DEFOG = df_clf_predictions_DEFOG[finalCols]

output_df = pd.concat([df_TDCS, df_DEFOG], ignore_index=True)
   
    
print('Writing submission file...')
output_df.to_csv("/kaggle/working/submission.csv", index=False)


Writing submission file...
